# Parse Trees and Probabilistic Context-Free Grammar

The goal of this program is, given treebank, generate a probabilistic context-free grammar using the frequency of rules used in the treebank.

## Reading the treebank

The input treebank will be specified in a format similar to `data/sample-corpus`, which contains the following sample treebank:

```
(S (NP /John/) (VP (VP (V /plays/) (NP /soccer/)) (PP (P /at/) (NP /school/))))
(S (NP /John/) (VP (VP (V /plays/) (NP /soccer/)) (PP (P /at/) (NP /school/))))
(S (NP /John/) (VP (VP (V /plays/) (NP /soccer/)) (PP (P /at/) (NP /school/))))
(S (NP /John/) (VP (V /plays/) (NP (NP /soccer/) (PP (P /at/) (NP /school/)))))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
(S (NP /John/) (VP (V /plays/) (NP /soccer/)))
```

The following code will read each line (tree) into a tree object, so it can be programatically analyzed.

First, we read each line of input.

In [1]:
with open('data/sample-corpus') as f:
    # We get rid of any / as we assume words do not contain /, 
    # and this makes parsing easier with nltk
    lines = [line.rstrip().replace("/","") for line in f]

We then convert each line (tree) into a tree object, using `nltk`.

In [2]:
from nltk import Tree
treeBank = [Tree.fromstring(line) for line in lines]

We can confirm this works by looking at the first line in our input treebank, and viewing its tree representation.

In [3]:
if not lines:
    print("No Input!")
else:
    print(lines[0])
    treeBank[0].pretty_print()

(S (NP John) (VP (VP (V plays) (NP soccer)) (PP (P at) (NP school))))
                 S                      
  _______________|_____                  
 |                     VP               
 |           __________|_______          
 |          VP                 PP       
 |      ____|____           ___|____     
 NP    V         NP        P        NP  
 |     |         |         |        |    
John plays     soccer      at     school



## Generating the Probablistic Context-Free Grammar

Using the `treeBank`, we can generate a Probablistic Context-Free Grammar (PCFG) with the following code:

In [4]:
nonTerminalCounts = {}
ruleCounts = {}

def parseTree(tree):    
    #tree.pretty_print()
    label = tree.label()
    
    # Parse each child
    childrenString = ""
    for child in tree:
        if isinstance(child, str):
            childrenString += f" {child}"
        else:
            childrenString += f" {child.label()}"
            parseTree(child)
    childrenString = childrenString[1:]
    
    # Update the count of this non-terminal
    if label in nonTerminalCounts:
        nonTerminalCounts[label] += 1
    else:
        nonTerminalCounts[label] = 1
    
    # Update the count of this rule
    rule = (label, childrenString)
    if rule in ruleCounts:
        ruleCounts[rule] += 1
    else:
        ruleCounts[rule] = 1

for tree in treeBank:
    #tree.pretty_print()
    parseTree(tree)

In [5]:
nonTerminalCounts

{'NP': 25, 'V': 10, 'VP': 13, 'P': 4, 'PP': 4, 'S': 10}

In [6]:
ruleCounts

{('NP', 'John'): 10,
 ('V', 'plays'): 10,
 ('NP', 'soccer'): 10,
 ('VP', 'V NP'): 10,
 ('P', 'at'): 4,
 ('NP', 'school'): 4,
 ('PP', 'P NP'): 4,
 ('VP', 'VP PP'): 3,
 ('S', 'NP VP'): 10,
 ('NP', 'NP PP'): 1}

Finally, we can produce our PCFG.

In [7]:
pcfg = {}
for rule, count in ruleCounts.items():
    pcfg[f"{rule[0]} -> {rule[1]}"] = count/nonTerminalCounts[rule[0]]
pcfg

{'NP -> John': 0.4,
 'V -> plays': 1.0,
 'NP -> soccer': 0.4,
 'VP -> V NP': 0.7692307692307693,
 'P -> at': 1.0,
 'NP -> school': 0.16,
 'PP -> P NP': 1.0,
 'VP -> VP PP': 0.23076923076923078,
 'S -> NP VP': 1.0,
 'NP -> NP PP': 0.04}